In [1]:
from __future__ import division
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn import preprocessing

In [2]:
def contBatActions2(x,demand,pv_gen,bat_t0,elprices,battery,const1,const2):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    lowBound=battery.capacity*0.2
    upBound=battery.capacity*0.8
    dim = len(x)
    bat_action = x[0]-bat_t0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == upBound:
                contChar+=1
                charOrDisch=-1                
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == lowBound:
                contDisch+=1
                charOrDisch=1                
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == upBound:
            charOrDisch=-1
        elif x[0] == lowBound:
            charOrDisch=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == upBound:
                    contChar+=1
                    charOrDisch=-1                
            elif bat_action < 0:
                charOrDisch=0
            
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == lowBound:
                    contDisch+=1
                    charOrDisch=1                
            elif bat_action > 0:
                charOrDisch=0
            
        elif charOrDisch == 0:
            if x[i] == upBound:
                charOrDisch=-1
            elif x[i] == lowBound:
                charOrDisch=1
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = const1*(grid_sum)-const2*(contChar+contDisch)
    return fitness, charOrDisch, contChar, contDisch, grid_sum

def contBatActions(x,demand,pv_gen,bat_t0,elprices,battery,const1,const2):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    lowBound=battery.capacity*0.2
    upBound=battery.capacity*0.8
    if x[0]-lowBound < 0.001:
        x[0]=lowBound
    if upBound-x[0] < 0.001:
        x[0]=upBound
    dim = len(x)
    bat_action = x[0]-bat_t0
    bat_action=float(bat_action)
    if abs(bat_action) <= 0.001:
        bat_action=0.0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == upBound:
                contChar+=1
                charOrDisch=-1                
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == lowBound:
                contDisch+=1
                charOrDisch=1                
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == upBound:
            charOrDisch=-1
        elif x[0] == lowBound:
            charOrDisch=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        bat_action=float(bat_action)
        if abs(bat_action) <= 0.001:
            bat_action=0.0
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == upBound:
                    contChar+=1
                    charOrDisch=-1                
            elif bat_action < 0:
                charOrDisch=0
            
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == lowBound:
                    contDisch+=1
                    charOrDisch=1                
            elif bat_action > 0:
                charOrDisch=0
            
        elif charOrDisch == 0:
            if x[i] == upBound:
                charOrDisch=-1
            elif x[i] == lowBound:
                charOrDisch=1
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = const1*(grid_sum)-const2*(contChar+contDisch)
    return fitness, charOrDisch, contChar, contDisch, grid_sum

def costFunctionBatteryLifetime3(x,demand,pv_gen,bat_t0,bat_action_t0,elprices,battery,const1,const2,const3):

    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    batActionTotal=0
    
    numCharges=0
    numDischarges=0
    batLifeLoss=0
    dischAction=0
    DOD=0
    dischFinish=0
    
    charOrDisch2 = battery.action
    contChar=0
    contDisch=0
    
    numCharChanges=0
    numBatNullState=0
    dim = len(x)
    bat_action = x[0]-bat_t0
    
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if bat_action == 0:
        numBatNullState+=1
        
    if charOrDisch2 == 1 and bat_action < 0:
        charOrDisch2=-1
        numCharChanges+=1 
        numDischarges+=1
        DOD=(battery.capacity-x[0])/battery.capacity #vypocet depth of discharge
        dischAction=bat_action
        dischFinish=0
        
    elif charOrDisch2 == -1 and bat_action < 0:
        DOD=(battery.capacity-x[0])/battery.capacity #vypocet depth of discharge
        dischAction=(bat_action_t0+bat_action)
        dischFinish=0
            
    elif charOrDisch2 == -1 and bat_action > 0:
        charOrDisch2=1
        numCharChanges+=1
        numCharges+=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    batActionTotal+=abs(bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
            
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if bat_action == 0:
            numBatNullState+=1
            
        if charOrDisch2 == 1 and bat_action < 0:
            charOrDisch2=-1
            numCharChanges+=1 
            numDischarges+=1
            DOD=(battery.capacity-x[i])/battery.capacity #vypocet depth of discharge
            dischAction=bat_action
            dischFinish=0
            
        elif charOrDisch2 == -1 and bat_action < 0:
            DOD=(battery.capacity-x[i])/battery.capacity #vypocet depth of discharge
            dischAction+=(bat_action)
            dischFinish=0
            
        elif charOrDisch2 == -1 and bat_action > 0:
            charOrDisch2=1
            numCharChanges+=1
            numCharges+=1
            batLifeLoss+=(abs(dischAction)*DOD)
            dischFinish=1
            
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
        batActionTotal+=abs(bat_action)

    if dischFinish==0:
        batLifeLoss+=(abs(dischAction)*DOD)
    if numDischarges>0:
        batLifeLoss=batLifeLoss
    else:
        batLifeLoss=0
    fitness = (const1*grid_sum)+(const2*numBatNullState)+(const3*batLifeLoss)
    return fitness, charOrDisch2, numCharges, numDischarges, grid_sum, numCharChanges, batLifeLoss

In [3]:
class Battery:
    def __init__(self,cap,soc,act,contAct,numContChar,numContDisch,numCharges,numDischarges,numBatNullState,numCharChanges):
        self.capacity=cap
        self.SOC=soc
        self.action=act
        self.contAct=contAct
        self.numContChar=numContChar
        self.numContDisch=numContDisch
        self.numCharges=numCharges
        self.numDischarges=numDischarges
        self.numBatNullState=numBatNullState
        self.numCharChanges=numCharChanges
        self.batLifeLoss=0
        self.avgDOD=0
        self.sumDOD=0
        self.avgDODPerDisch=0
        self.sumDODPerDisch=0
        self.avgSOC=0
        self.sumSOC=0
        self.avgDischAction=0
        self.sumDischAction=0
        self.totalBat=0

In [4]:
class BatAlgorithm2():
    def __init__(self, D, NP, N_Gen, A, r, Qmin, Qmax, Lower, Upper,function,demand,pv_gen,bat_t0,bat_action_t0,elprices,battery,const1,const2,const3):
        self.D = D  #dimension
        self.NP = NP  #population size 
        self.N_Gen = N_Gen  #generations
        self.A = A  #loudness
        self.r = r  #pulse rate
        self.Qmin = Qmin  #frequency min
        self.Qmax = Qmax  #frequency max
        self.Lower = Lower  #lower bound
        self.Upper = Upper  #upper bound
        
        self.battery = battery    #pridal
        self.pvToDemand=0         #pridal
        self.const1 = const1      #pridal
        self.const2 = const2      #pridal
        self.const3 = const3      #pridal

        self.f_min = 0.0  #minimum fitness
        
        self.charOrDischBest = 0   #pridal
        self.contCharBest = 0      #pridal
        self.contDischBest = 0     #pridal
        self.gridPriceBest = 0     #pridal
        
        self.charOrDisch = [0] * self.NP #pridal
        self.contChar = [0] * self.NP    #pridal
        self.contDisch = [0] * self.NP   #pridal
        self.gridPrice = [0] * self.NP   #pridal
        
        self.charOrDisch2Best = 0    #pridal
        self.numBatNullStateBest = 0 #pridal
        self.numCharChangesBest = 0  #pridal
        self.numChargesBest = 0      #pridal
        self.numDischargesBest = 0   #pridal
        
        self.charOrDisch2 = [0] * self.NP      #pridal
        self.numBatNullState = [0] * self.NP   #pridal
        self.numCharChanges = [0] * self.NP    #pridal
        self.numCharges = [0] * self.NP        #pridal
        self.numDischarges = [0] * self.NP     #pridal
        
        self.Lb = [0] * self.D  #lower bound
        self.Ub = [0] * self.D  #upper bound
        self.Q = [0] * self.NP  #frequency

        self.v = [[0 for i in range(self.D)] for j in range(self.NP)]  #velocity
        self.Sol = [[0 for i in range(self.D)] for j in range(self.NP)]  #population of solutions
        self.Fitness = [0] * self.NP  #fitness
        self.best = [0] * self.D  #best solution
        self.Fun = function

        
        self.demand = demand       #pridal
        self.pv_gen = pv_gen       #pridal
        self.elprices = elprices   #pridal
        self.bat_t0 = bat_t0       #pridal
        self.bat_action_t0 = bat_action_t0 #pridal
        self.grid_actions = []     #pridal
        self.demand_sum = 0        #pridal
        self.grid_total = 0        #pridal


    def best_bat(self):
        i = 0
        j = 0
        for i in range(self.NP):
            if self.Fitness[i] < self.Fitness[j]:
                j = i
        for i in range(self.D):
            self.best[i] = self.Sol[j][i]
        self.f_min = self.Fitness[j]

    def init_bat(self):
        for i in range(self.D):
            self.Lb[i] = self.Lower
            self.Ub[i] = self.Upper

        for i in range(self.NP):
            self.Q[i] = 0
            for j in range(self.D):
                rnd = np.random.uniform(0, 1)
                self.v[i][j] = 0.0
                self.Sol[i][j] = self.Lb[j] + (self.Ub[j] - self.Lb[j]) * rnd

            self.Fitness[i], self.charOrDisch2[i], self.numCharges[i], self.numDischarges[i], self.gridPrice[i], self.numCharChanges[i], self.numBatNullState[i] = self.Fun(self.Sol[i],self.demand,self.pv_gen,self.bat_t0,self.bat_action_t0,self.elprices,self.battery,self.const1,self.const2,self.const3)
            
        self.best_bat()

    def simplebounds(self, val, lower, upper):
        if val < lower:
            val = lower
        if val > upper:
            val = upper
        return val

    def move_bat(self):
        S = [[0.0 for i in range(self.D)] for j in range(self.NP)]

        self.init_bat()

        for t in range(self.N_Gen):
            for i in range(self.NP):
                rnd = np.random.uniform(0, 1)
                self.Q[i] = self.Qmin + (self.Qmax - self.Qmin) * rnd
                for j in range(self.D):
                    self.v[i][j] = self.v[i][j] + (self.Sol[i][j] -
                                                   self.best[j]) * self.Q[i]
                    S[i][j] = self.Sol[i][j] + self.v[i][j]

                    S[i][j] = self.simplebounds(S[i][j], self.Lb[j],
                                                self.Ub[j])

                rnd = np.random.random_sample()

                if rnd > self.r:
                    for j in range(self.D):
                        S[i][j] = self.best[j] + 0.001 * random.gauss(0, 1)
                        S[i][j] = self.simplebounds(S[i][j], self.Lb[j],
                                                self.Ub[j])
                        

                Fnew, charOrDisch2New, numChargesNew, numDischargesNew, gridPriceNew, numCharChangesNew, numBatNullStateNew = self.Fun(S[i],self.demand,self.pv_gen,self.bat_t0,self.bat_action_t0,self.elprices,self.battery,self.const1,self.const2,self.const3)
                
                rnd = np.random.random_sample()

                if (Fnew <= self.Fitness[i]) and (rnd < self.A):
                    for j in range(self.D):
                        self.Sol[i][j] = S[i][j]
                    self.Fitness[i] = Fnew

                if Fnew <= self.f_min:
                    for j in range(self.D):
                        self.best[j] = S[i][j]
                    self.f_min = Fnew
                    self.charOrDischBest = 0 #pridal
                    self.contCharBest = 0    #pridal
                    self.contDischBest = 0   #pridal
                    self.gridPriceBest = 0   #pridal
                    
                    self.charOrDisch2Best = charOrDisch2New      #pridal
                    self.numChargesBest =  numChargesNew         #pridal
                    self.numDischargesBest =  numDischargesNew   #pridal
                    self.numCharChangesBest = numCharChangesNew  #pridal
                    self.numBatNullStateBest = numBatNullStateNew#pridal
                    

       

In [7]:
class BAOptimizer():
    def __init__(self, Nparticles, Niter, start, end, windowSize, const1, const2, const3, batCapacity, batSOC, houseNumber):
        global str
        self.Nparticles=Nparticles
        self.Niter=Niter
        self.houseNumber=houseNumber
        self.totalImport = 0
        self.totalExport = 0
        self.start = start
        self.end = end
        self.windowSize=windowSize
        self.bat=Battery(cap=batCapacity, soc=batSOC, act=1, contAct=0, numContChar=0, numContDisch=0, numCharges=1, numDischarges=0, numBatNullState=0, numCharChanges=0)
        self.const1=const1
        self.const2=const2
        self.const3=const3
        
        #polia nakladania s energiou
        self.gridTotalProgress = []
        self.pvToDemandProgress = []
        self.batSOCProgress = []
        self.batActionsProgress = []
        
        self.totalPrice=0
        self.total_demand=0
        self.pvToDemandTotal=0
        
        self.data = pd.read_csv('../data/load/dataid' + str(houseNumber) + '.csv', index_col=0, sep=";")
        self.data.loc[self.data['loadProd'] < 0, 'loadProd'] = 0
        
        self.elprices = pd.read_csv('../data/elprices.csv', index_col=0, sep=",")
        self.elprices = pd.DataFrame(self.elprices[['prices']])
        self.elprices = self.elprices['prices']

        self.iter=0
        self.gen = pd.DataFrame(self.data[['loadProd']])
        self.con = pd.DataFrame(self.data[['loadCons']])
        self.con = self.con['loadCons']
        self.gen = self.gen['loadProd']

    def calculateImportExport(self,gridAction):
        if gridAction >= 0:
            self.totalImport+=gridAction
        if gridAction < 0:
            self.totalExport+=(-gridAction)
            
    def calculatePrice(self,elprices,gridActions):
        price=0
        for i in range(0,len(elprices)):
            if gridActions[i]>0:
                price+=(elprices[i]*gridActions[i])
        return price
    
    def saveResults(self,gridTotalProgress,batSOCProgress,batActionsProgress,pvToDemandProgress,total_demand, pvToDemandTotal,contAct,numContChar,numContDisch,totalPrice,finalnumCharges,finalnumDischarges,finalnumCharChanges,finalnumNullStates):    
        df=self.data[self.start-1:self.end]
        batSoc = batSOCProgress
        if self.iter==0:
            batSoc.pop(0)
            self.iter=1
        dat1 = pd.DataFrame({'newLoadGrid': gridTotalProgress})
        dat2 = pd.DataFrame({'batterySOC': batSoc})
        dat3 = pd.DataFrame({'batteryActions': batActionsProgress})
        dat4 = pd.DataFrame({'pvUsage': pvToDemandProgress})
        dat5 = pd.DataFrame({'prices': self.elprices[self.start-1:self.end]})
        dat5.index = np.arange(0, len(dat5) + 0)
        dat6 = pd.DataFrame({'batteryLastSOC': [self.bat.SOC]})
        dat7 = pd.DataFrame({'totalPrice': [totalPrice]})
        dat8 = pd.DataFrame({'totalImport': [self.totalImport]})
        dat9 = pd.DataFrame({'totalExport': [self.totalExport]})
        dat10 = pd.DataFrame({'totalCons': [total_demand]})
        dat11 = pd.DataFrame({'totalPvUsage': [pvToDemandTotal]})
        dat12 = pd.DataFrame({'contAct': [contAct]})
        dat26 = pd.DataFrame({'batteryAct': [self.bat.action]})
        dat13 = pd.DataFrame({'numContChar': [numContChar]})
        dat14 = pd.DataFrame({'numContDisch': [numContDisch]})
        
        dat15 = pd.DataFrame({'charges': [finalnumCharges]})
        dat16 = pd.DataFrame({'discharges': [finalnumDischarges]})
        dat17 = pd.DataFrame({'CharChanges': [finalnumCharChanges]})
        dat18 = pd.DataFrame({'batNullStates': [finalnumNullStates]})
        
        dat19 = pd.DataFrame({'batLifeLoss': [self.bat.batLifeLoss]})
        if finalnumDischarges==0:
            dat20 = pd.DataFrame({'batLifeLossPerDisch': [0]})
        else:
            dat20 = pd.DataFrame({'batLifeLossPerDisch': [self.bat.batLifeLoss/finalnumDischarges]})
        dat21 = pd.DataFrame({'avgDOD': [self.bat.avgDOD]})
        dat22 = pd.DataFrame({'avgDODPerDisch': [self.bat.avgDODPerDisch]})
        dat23 = pd.DataFrame({'avgDischAction': [self.bat.avgDischAction]})
        dat24 = pd.DataFrame({'DischAction': [self.bat.avgDischAction*finalnumDischarges]})
        dat25 = pd.DataFrame({'avgSOC': [self.bat.avgSOC]})
        dat27 = pd.DataFrame({'totalBat': [self.bat.totalBat]})
        
        dat28 = pd.DataFrame({'sumDOD': [self.bat.sumDOD]})
        dat29 = pd.DataFrame({'sumDODPerDisch': [self.bat.sumDODPerDisch]})
        dat30 = pd.DataFrame({'sumSOC': [self.bat.sumSOC]})
        dat31 = pd.DataFrame({'sumDischAction': [self.bat.sumDischAction]})
        
        dat1 = pd.concat([dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8, dat9, dat10,dat11, dat12, dat26, dat13, dat14, dat15, dat16, dat17, dat18, dat19, dat20, dat21, dat22, dat23, dat24, dat25, dat27, dat28, dat29, dat30, dat31], axis=1)
        dat1.index = np.arange(self.start, len(dat1) + self.start)
        df = pd.concat([df, dat1], axis=1)
        df.to_csv("Vysledky/BAMaxBatLife/BAMaxBatLifeData"+str(self.houseNumber)+".csv",sep=";")
        
    def calculateContCharDisch(self,batSOCProgress,batSoc,bat):
        conWindow=[]
        genWindow=[]
        elprices=[]
        for i in range(self.start,self.end+1):
            conWindow.append(self.con[i])
            genWindow.append(self.gen[i])
            elprices.append(self.elprices[i])
        vvar, vcontAct, vnumContChar,vnumContDisch, vvar3 = contBatActions(batSOCProgress,conWindow,genWindow,batSoc,elprices,bat,self.const1, self.const2)
        self.bat.numContChar = vnumContChar
        self.bat.numContDisch = vnumContDisch
        print('cont actions ',vcontAct, vnumContChar,vnumContDisch)
            
    def calculateBatteryChargingActions(self, Algorithm, conWindow, genWindow, elprices):
        #vypocet kontinualnych nabiti
        var1, charOrDisch, numContChar, numContDisch, var3 = contBatActions2(Algorithm.best,conWindow, genWindow, self.bat.SOC, elprices,self.bat,self.const1, self.const2)
        self.bat.contAct=charOrDisch
            
        self.bat.action=Algorithm.charOrDisch2Best
        self.bat.numCharChanges+=Algorithm.numCharChangesBest   
        self.bat.numCharges+=Algorithm.numChargesBest
        self.bat.numDischarges+=Algorithm.numDischargesBest
        
    def calculateBatteryLossActions(self,batActionsProgress,batSOCProgress,charOrDisch):
        numCharChanges = 0
        numCharges = 1
        numDischarges = 0
        numNullStates=0
        batLifeLoss=0
        dischAction=0
        dischFinish=0
        DODProgress=[]
        DODProgress2=[]
        DOD=0
        avgDOD=0
        avgDOD2=0
        avgSOC=0
        avgDischAction=0
        length = len(batActionsProgress)
        for i in range(0,length):
            batAction=batActionsProgress[i]
            avgSOC+=(batSOCProgress[i]/self.bat.capacity)
            avgDOD2+=(self.bat.capacity-batSOCProgress[i])
            DODProgress2.append((self.bat.capacity-batSOCProgress[i]))
            
            if charOrDisch==1 and batAction < 0:
                numCharChanges+=1
                numDischarges+=1
                charOrDisch=-1
                DOD=(self.bat.capacity-batSOCProgress[i])/self.bat.capacity #vypocet depth of discharge
                dischAction=batAction
                dischFinish=0
                
            elif charOrDisch == -1 and batAction < 0:
                DOD=(self.bat.capacity-batSOCProgress[i])/self.bat.capacity #vypocet depth of discharge
                dischAction+=(batAction)
                dischFinish=0
                
            elif charOrDisch==-1 and batAction > 0:
                numCharChanges+=1
                numCharges+=1
                charOrDisch=1
                batLifeLoss+=(abs(dischAction)*DOD)
                avgDischAction+=abs(dischAction)
                avgDOD+=DOD
                DODProgress.append(DOD)
                dischFinish=1
                
            elif batAction == 0:
                numNullStates+=1
                
        if dischFinish==0:
            batLifeLoss+=(abs(dischAction)*DOD)
            avgDOD+=DOD
            DODProgress.append(DOD)
        if numDischarges>0:
            batLifeLoss=batLifeLoss/numDischarges
            avgDOD=avgDOD/numDischarges
            avgDischAction=avgDischAction/numDischarges
        else:
            batLifeLoss=0
        print("numCharChanges ",numCharChanges)
        print("numCharges ",numCharges)
        print("numDischarges ",numDischarges)
        print("numNullStates ",numNullStates)
        self.bat.numBatNullState=numNullStates
        print("batLifeLoss ",batLifeLoss*numDischarges)
        #print("avgDOD ",(sum(DODProgress)/len(DODProgress)),len(DODProgress))
        print("avgDOD ",(sum(DODProgress2)/len(DODProgress2))/self.bat.capacity)
        print("avgSOC ",(sum(batSOCProgress)/len(batSOCProgress))/self.bat.capacity)
        print("avgDischAction ",avgDischAction*numDischarges)
        
        totalBat=0
        for i in batActionsProgress:
            totalBat+=abs(i)
        print('totalBat: ',totalBat)
        
        self.bat.batLifeLoss=batLifeLoss*numDischarges
        self.bat.avgDOD=(sum(DODProgress2)/len(DODProgress2))/self.bat.capacity
        self.bat.avgDODPerDisch=(sum(DODProgress)/len(DODProgress))
        self.bat.avgSOC=(sum(batSOCProgress)/len(batSOCProgress))/self.bat.capacity
        self.bat.avgDischAction=avgDischAction
        self.bat.totalBat=totalBat
        
        self.bat.sumDOD=sum(DODProgress2)
        self.bat.sumDODPerDisch=sum(DODProgress)
        self.bat.sumSOC=sum(batSOCProgress)
        self.bat.sumDischAction=avgDischAction*numDischarges
        
    def optimize(self):
        print("OPTIMALIZACIA particles x iter: ",self.Nparticles,self.Niter)
        windowSize=self.windowSize
        start=self.start
        end = self.end
        
        total_grid = 0
        total_price = 0
        batteryAction=0

        
        self.batSOCProgress.append(0)
        import time
        zac = int(round(time.time() * 1000))
        print(zac)

        for c in range(start,end,windowSize):
            d=c+windowSize
            initial= []
            genWindow = []
            conWindow = []
            BSSBounds = []
            elprices = []
            print("row number: ",c)
            for i in range(c,d):
                if i<=end:
                    elprices.append(self.elprices[i])
                    genWindow.append(self.gen[i])
                    conWindow.append(self.con[i])
                    BSSBounds.append((0,self.bat.capacity))
                    if i % 24 == 0:
                        print(i)     
            Algorithm = BatAlgorithm2(len(genWindow), self.Nparticles, self.Niter, 0.5, 0.5, 0.0, 2.0, self.bat.capacity*0.2, self.bat.capacity*0.8, costFunctionBatteryLifetime3, conWindow, genWindow, self.bat.SOC, batteryAction, elprices, self.bat, self.const1, self.const2, self.const3)
            Algorithm.move_bat()

            #prerobene     
            alg_bat_actions = []                                                
            alg_grid_actions = []
            alg_bat_actions.append(Algorithm.best[0]-self.bat.SOC)
            
            batteryAction=Algorithm.best[0]-self.bat.SOC
            self.batActionsProgress.append(batteryAction)
            
            gridActionHour = (conWindow[0]-genWindow[0]+alg_bat_actions[0])
            alg_grid_actions.append(gridActionHour)
            self.calculateImportExport(gridActionHour)
            alg_demand_sum = conWindow[0]
            alg_grid_total = alg_grid_actions[0]
            for k in range(1,Algorithm.D):
                alg_bat_actions.append(Algorithm.best[k]-Algorithm.best[k-1])
                self.batActionsProgress.append(Algorithm.best[k]-Algorithm.best[k-1])
                gridActionHour=(conWindow[k]-genWindow[k]+alg_bat_actions[k])
                alg_grid_actions.append(gridActionHour)
                self.calculateImportExport(gridActionHour)
                alg_demand_sum+=conWindow[k]
                alg_grid_total+=alg_grid_actions[k]
            #prerobene
            
            #vypocet spotreby z pv   
            algPvToDemand=0
            for i in range(0,Algorithm.D):
                pvRest=0
                algPvToDemandInHour = 0
                if genWindow[i] >= conWindow[i]:
                    algPvToDemand += conWindow[i]
                    algPvToDemandInHour += conWindow[i]
                    pvRest = genWindow[i]-conWindow[i]
                else:
                    algPvToDemand += genWindow[i]
                    algPvToDemandInHour += genWindow[i]
                if pvRest > 0 and alg_bat_actions[i]>0:
                    if alg_bat_actions[i]>=pvRest:
                        algPvToDemand += pvRest
                        algPvToDemandInHour += pvRest
                    else:
                        algPvToDemand += alg_bat_actions[i]
                        algPvToDemandInHour += alg_bat_actions[i]
                self.pvToDemandProgress.append(algPvToDemandInHour)
            #vypocet spotreby z pv
            
            self.bat.SOC=Algorithm.best[Algorithm.D-1]

            l = len(alg_grid_actions)
            for j in range(0,l):
                self.gridTotalProgress.append(alg_grid_actions[j])
                self.batSOCProgress.append(Algorithm.best[j])
            self.total_demand += alg_demand_sum#prerobene
            total_grid += alg_grid_total#prerobene
            price = self.calculatePrice(elprices,alg_grid_actions)
            self.totalPrice += price
            self.pvToDemandTotal+=algPvToDemand
            
            print()
           
            self.calculateBatteryChargingActions(Algorithm, conWindow, genWindow, elprices)
            self.calculateContCharDisch(self.batSOCProgress,self.bat.SOC,self.bat)
            self.calculateBatteryLossActions(self.batActionsProgress,self.batSOCProgress,1)
            self.saveResults(self.gridTotalProgress, self.batSOCProgress, self.batActionsProgress, self.pvToDemandProgress,self.total_demand,self.pvToDemandTotal,self.bat.contAct,self.bat.numContChar,self.bat.numContDisch,self.totalPrice,self.bat.numCharges,self.bat.numDischarges,self.bat.numCharChanges,self.bat.numBatNullState)

        print()
        print('FINAL:')
        kon = int(round(time.time() * 1000))
        #print((kon/1000)/60)
        print("minut: ",((kon-zac)/1000)/60)
        print('Total demand: ',self.total_demand)
        print('Total price: ',self.totalPrice)
        print('Total Import: ',self.totalImport)
        print('Total Export: ',self.totalExport)
        print('pv to demand: ',self.pvToDemandTotal)
        print()
                
        
        
        print()
        print("Charging cycles")
        #self.calculateContCharDisch(self.batSOCProgress,self.bat.SOC,self.bat)
        
        self.calculateBatteryLossActions(self.batActionsProgress,self.batSOCProgress,1)

In [6]:
ba = BAOptimizer(Nparticles=13000,Niter=200,start=1,end=8760,windowSize=24,const1=1 ,const2=0, const3=1.5,batCapacity=13,batSOC=0,houseNumber=1642)
ba.optimize()

OPTIMALIZACIA particles x iter:  130 200
1590878619368
row number:  1
24

cont actions  0 0 0
numCharChanges  11
numCharges  6
numDischarges  6
numNullStates  0
batLifeLoss  1.9479134615706837
avgDOD  0.5250381157502313
avgSOC  0.4756283097081591
avgDischAction  3.7997387266299087
totalBat:  14.558179323268295
row number:  25
48

cont actions  0 0 0
numCharChanges  18
numCharges  10
numDischarges  9
numNullStates  0
batLifeLoss  4.083737976078521
avgDOD  0.5447171574976736
avgSOC  0.4552828425023265
avgDischAction  7.237373497045397
totalBat:  20.35729057265385
row number:  49
72

cont actions  0 0 0
numCharChanges  28
numCharges  15
numDischarges  14
numNullStates  0
batLifeLoss  6.833391620639635
avgDOD  0.531923815685931
avgSOC  0.4680761843140692
avgDischAction  11.996193460029986
totalBat:  31.574588806051075
row number:  73

cont actions  0 0 0
numCharChanges  32
numCharges  17
numDischarges  16
numNullStates  0
batLifeLoss  8.27746067468
avgDOD  0.5281171642233736
avgSOC  0.4718